In [1]:
import math

# Constantes del problema físico
N = 2
G = 1.0

# Constantes del algoritmo de integración
xi = 0.1786178958448091
lambda_val = -0.2123418310626054
chi = -0.06626458266981849
Um2lambdau2 = (1 - 2 * lambda_val) / 2
Um2chiplusxi = 1 - 2 * (chi + xi)

# Declarar la clase Vector3D
class Vector3D:
    def __init__(self, x, y, z):
        self.x_val = x
        self.y_val = y
        self.z_val = z

    def load(self, x, y, z):
        self.x_val = x
        self.y_val = y
        self.z_val = z

    def norm2(self):
        return self.x_val**2 + self.y_val**2 + self.z_val**2

# Declarar la clase Cuerpo
class Cuerpo:
    def __init__(self):
        self.r = Vector3D(0, 0, 0)
        self.V = Vector3D(0, 0, 0)
        self.F = Vector3D(0, 0, 0)
        self.m = 0
        self.R = 0

    def inicie(self, x0, y0, z0, Vx0, Vy0, Vz0, m0, R0):
        self.r.load(x0, y0, z0)
        self.V.load(Vx0, Vy0, Vz0)
        self.m = m0
        self.R = R0

    def borre_fuerza(self):
        self.F.load(0, 0, 0)

    def sume_fuerza(self, dF):
        self.F.x_val += dF.x_val
        self.F.y_val += dF.y_val
        self.F.z_val += dF.z_val

    def mueva_r(self, dt, coeficiente):
        self.r.x_val += self.V.x_val * (coeficiente * dt)
        self.r.y_val += self.V.y_val * (coeficiente * dt)
        self.r.z_val += self.V.z_val * (coeficiente * dt)

    def mueva_V(self, dt, coeficiente):
        self.V.x_val += self.F.x_val * (coeficiente * dt / self.m)
        self.V.y_val += self.F.y_val * (coeficiente * dt / self.m)
        self.V.z_val += self.F.z_val * (coeficiente * dt / self.m)

    def dibujese(self):
        print(", {0}+{1}*cos(t), {2}+{3}*sin(t)".format(self.r.x_val, self.R, self.r.y_val, self.R))

    def get_x(self):
        return self.r.x_val

    def get_y(self):
        return self.r.y_val


# Declarar la clase Colisionador
class Colisionador:
    def calcule_todas_las_fuerzas(self, planetas):
        for i in range(N):
            planetas[i].borre_fuerza()

        for i in range(N):
            for j in range(i):
                self.calcule_fuerza_entre(planetas[i], planetas[j])

    def calcule_fuerza_entre(self, planeta1, planeta2):
        m1, m2 = planeta1.m, planeta2.m
        r21 = Vector3D(planeta2.r.x_val - planeta1.r.x_val,
                       planeta2.r.y_val - planeta1.r.y_val,
                       planeta2.r.z_val - planeta1.r.z_val)
        r2 = r21.norm2()
        aux = G * m2 * m1 * r2**(-1.5)
        F1 = Vector3D(r21.x_val * aux, r21.y_val * aux, r21.z_val * aux)
        planeta1.sume_fuerza(F1)
        planeta2.sume_fuerza(Vector3D(-F1.x_val, -F1.y_val, -F1.z_val))


# Funciones de animación
def inicie_animacion():
    print("unset key")
    print("set xrange[-11:11]")
    print("set yrange[-11:11]")
    print("set size ratio -1")
    print("set parametric")
    print("set trange [0:7]")
    print("set isosamples 12")


def inicie_cuadro():
    print("plot 0,0", end=' ')


def termine_cuadro():
    print()


def main():
    r = 11
    m0, m1 = 10, 1
    M = m0 + m1
    mu = m0 * m1 / M
    x0 = -m1 * r / M
    x1 = m0 * r / M
    omega = math.sqrt(G * M / (r**3))
    T = 2 * math.pi / omega
    V0 = omega * x0
    V1 = omega * x1
    t, dt = 0, 0.1
    ttotal = T
    Ncuadros = 30000
    tdibujo, tcuadro = 0, ttotal / Ncuadros

    planetas = [Cuerpo() for _ in range(N)]
    newton = Colisionador()

    # InicieAnimacion()

    # INICIO
    planetas[0].inicie(x0, 0, 0, 0, 0.5 * V0, 0, m0, 1.0)
    planetas[1].inicie(x1, 0, 0, 0, 0.5 * V1, 0, m1, 0.5)

    # CORRO
    while t < ttotal:
        # if tdibujo > tcuadro:
        #     InicieCuadro()
        #     for i in range(N):
        #         Planeta[i].Dibujese()
        #     TermineCuadro()
        #     tdibujo = 0

        print(planetas[1].get_x(), planetas[1].get_y())

        for i in range(N):
            planetas[i].mueva_r(dt, xi)
        newton.calcule_todas_las_fuerzas(planetas)
        for i in range(N):
            planetas[i].mueva_V(dt, Um2lambdau2)

        for i in range(N):
            planetas[i].mueva_r(dt, chi)
        newton.calcule_todas_las_fuerzas(planetas)
        for i in range(N):
            planetas[i].mueva_V(dt, lambda_val)

        for i in range(N):
            planetas[i].mueva_r(dt, Um2chiplusxi)
        newton.calc

